#Text Summarization using T5 Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00


In [ ]:
!pip install lightning
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import contractions
import torch
import json
import lightning as pl

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from lightning.pytorch import Trainer
from transformers import AdamW, T5ForConditionalGeneration, T5TokenizerFast as T5Tokenizer
from tqdm.auto import tqdm

In [ ]:
pl.seed_everything(100)

INFO: Global seed set to 100
INFO:lightning.fabric.utilities.seed:Global seed set to 100


100

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DATASETS/news_summary.csv', encoding="latin-1")
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
df = df[['text', 'ctext']]
df.head()

,text,ctext
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
df.columns = ['summary', 'text']
df = df.dropna()
df = df.drop_duplicates()
df.head()

,summary,text
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
# Converting all the strings to lowercase in 'summary' and 'text' columns
df['text'] = df['text'].str.lower()
df['summary'] = df['summary'].str.lower()

In [ ]:
# expand contractions in the 'summary' and 'text' columns
df['summary'] = df['summary'].apply(lambda x: contractions.fix(x))
df['text'] = df['text'].apply(lambda x: contractions.fix(x))

In [ ]:
# remove punctuation marks from the 'summary' and 'text' columns
df['summary'] = df['summary'].str.replace('[^\w\s]','')
df['text'] = df['text'].str.replace('[^\w\s]','')

<ipython-input-12-5e7931a312d1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['summary'] = df['summary'].str.replace('[^\w\s]','')
<ipython-input-12-5e7931a312d1>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]','')


In [ ]:
df.shape

(4396, 2)

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1)
print(f"Shape of the Training Set: {train_df.shape}\nShape of the Testing Set: {test_df.shape}")

Shape of the Training Set: (3956, 2)
Shape of the Testing Set: (440, 2)


In [ ]:
class NewsDataset(Dataset):
  def __init__(self, data, tokenizer, text_max_token_len=512, summary_max_token_len=128):
    self.tokenizer = tokenizer
    self.data = data
    self.text_max_token_len = text_max_token_len
    self.summary_max_token_len = summary_max_token_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    data_row = self.data.iloc[index]
    text = data_row["text"]

    # Encode the text
    text_encoding = self.tokenizer(
        text,
        max_length=self.text_max_token_len,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )

    # Encode the summary
    summary_encoding = self.tokenizer(
        data_row["summary"],
        max_length=self.summary_max_token_len,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )

    # Modify the labels so that the model knows which tokens to predict
    labels = summary_encoding['input_ids']
    labels[labels == 0] = -100

    return {
        'text': text,
        'summary': data_row['summary'],
        'text_input_ids': text_encoding['input_ids'].flatten(),
        'text_attention_mask': text_encoding['attention_mask'].flatten(),
        'labels': labels.flatten(),
        'labels_attention_mask': summary_encoding["attention_mask"].flatten()
    }


In [ ]:
class NewsDataModule(pl.LightningDataModule):
  def __init__(self, train_df, test_df, tokenizer, batch_size=8, text_max_token_len=152, summary_max_token_len=128):
    super().__init__()

    self.train_df = train_df
    self.test_df = test_df

    self.batch_size = batch_size
    self.tokenizer = tokenizer
    self.text_max_token_len = text_max_token_len
    self.summary_max_token_len = summary_max_token_len

  def setup(self, stage=None):

    self.train_dataset = NewsDataset(
        self.train_df,
        self.tokenizer,
        self.text_max_token_len,
        self.summary_max_token_len)

    self.test_dataset = NewsDataset(
        self.test_df,
        self.tokenizer,
        self.text_max_token_len,
        self.summary_max_token_len)

  def train_dataloader(self):

    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True
    )

  def test_dataloader(self):

    return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        shuffle=False
    )

  def val_dataloader(self):

    return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        shuffle=False
    )

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
n_epochs= 2
BATCH_SIZE=10

data_module = NewsDataModule(
    train_df,
    test_df,
    tokenizer,
    batch_size=BATCH_SIZE

)

Model

In [ ]:
class SummaryModel(pl.LightningModule):
    def __init__(self):
      super().__init__()
      self.model = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)

    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
      output = self.model(
        input_ids,
        attention_mask=attention_mask,
        labels=labels,
        decoder_attention_mask=decoder_attention_mask
        )
      return output.loss, output.logits

    def shared_step(self, batch, batch_idx, stage):
        input_ids = batch['text_input_ids']
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, _ = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels
        )

        return loss

    def training_step(self, batch, batch_idx):
        return self.shared_step(batch, batch_idx, 'train')

    def validation_step(self, batch, batch_idx):
        return self.shared_step(batch, batch_idx, 'val')

    def test_step(self, batch, batch_idx):
        return self.shared_step(batch, batch_idx, 'test')

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0003)


In [ ]:
model_1 = SummaryModel()

In [ ]:
trainer= Trainer(
  max_epochs=n_epochs
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model_1, data_module)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO: 
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
---------------------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=2` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


In [ ]:
def encode_text(text):
  # Encode the text using the tokenizer
  encoding = tokenizer.encode_plus(
      text,
      max_length=512,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  return encoding["input_ids"], encoding["attention_mask"]

def generate_summary(input_ids, attention_mask):
  # Generate a summary using the model
  generated_ids = model_1.model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_length=150,
      num_beams=2,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True
  )
  return generated_ids

def decode_summary(generated_ids):
  # Decode the generated summary
  summary = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
              for gen_id in generated_ids]
  return "".join(summary)

def summarize(text):
  input_ids, attention_mask = encode_text(text)
  generated_ids = generate_summary(input_ids, attention_mask)
  summary = decode_summary(generated_ids)
  return summary


In [ ]:
sample = test_df.iloc[10]
predicted_summary = summarize(sample["text"])

In [ ]:
sample["text"]

'swashbuckling opener chris gayle has been included in the 13man west indies team which will play a t20 international match against india in jamaica on sunday july 9the lefthander is the highest runmaker for the windies in the t20 format with 1519 runs average 3532 strike rate 14549 including two centuries this will be his first t20 international on his home ground sabina park he last played a t20 international for west indies in april 2016 in the world t20 final vs england at kolkatas eden gardenswe welcome chris back to the t20 squad he is the most prolific batsman in this format and will add value to our team at the top of the order he will get the chance to play on his home ground and against a topquality indian team said courtney browne cricket west indies chairman of selectorsthe lefthanded batsman has been included in the side at the expense of lendl simmons who scored 6 17 and 15 in the series against afghanistan west indies last t20 assignmenttest and odi skipper jason holder 

In [ ]:
sample["summary"]

'windies highest run scorer in t20is chris gayle has been included in the 13man squad for the oneoff t20i against india marking his return to the international circuit after a 15month gap gayles last international match was the world t20 final against england in april 2016 the t20i match between india and windies will be held on july 9'

In [ ]:
predicted_summary

'swashbuckling opener chris gayle has been included in the 13man west indies squad which will play a t20 match against india in jamaica on july 9 this will be his first t20 international at his home ground and will add value to our team said courtney browne'

In [ ]:
!pip install rouge
from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(predicted_summary, sample["summary"])

rouge1_score = scores[0]["rouge-1"]["f"]
rouge2_score = scores[0]["rouge-2"]["f"]
rougeL_score = scores[0]["rouge-l"]["f"]

print("ROUGE-1 score:", rouge1_score)
print("ROUGE-2 score:", rouge2_score)
print("ROUGE-L score:", rougeL_score)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ROUGE-1 score: 0.5168539276000507
ROUGE-2 score: 0.2115384566050297
ROUGE-L score: 0.35955055681353365
